In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from pandas import DataFrame
import csv
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import collections

pd.options.display.max_rows

pd.options.display.max_columns

20

In [2]:
#function to lag

def buildLaggedFeatures(s,columns, lag=5,dropna=True): #lag=3
    '''
    From http://stackoverflow.com/questions/20410312/how-to-create-a-lagged-data-structure-using-pandas-dataframe
    Builds a new DataFrame to facilitate regressing over all possible lagged features
    '''
    if type(s) is pd.DataFrame:
        new_dict={}
        for c in s.columns:
            new_dict[c]=s[c]
        for col_name in columns:
            new_dict[col_name]=s[col_name]
            # create lagged Series
            for l in range(1,lag+1):
                new_dict['%s_lag%d' %(col_name,l)]=s[col_name].shift(l)
        res=pd.DataFrame(new_dict,index=s.index)

    elif type(s) is pd.Series:
        the_range=range(lag+1)
        res=pd.concat([s.shift(i) for i in the_range],axis=1)
        res.columns=['lag_%d' %i for i in the_range]
    else:
        print('Only works for DataFrame or Series')
        return None
    if dropna:
        return res.dropna()
    else:
        return res 


#function to set in each row if there is an event in the 4h before 
    


def check_ev_diff(ev,df):
    for i in range(200):

        print("---------------------------------------------------")
        print(len(df))

        df.reset_index(drop=True, inplace=True)

        for bo_index, bo_row in df.iterrows():
            for e_index, e_row in ev.iterrows():
                if e_row.starting_time.day == bo_row.TimeStep.day and e_row.starting_time.month == bo_row.TimeStep.month and e_row.starting_time.year == bo_row.TimeStep.year:

                    diff= e_row.starting_time - bo_row.TimeStep
                    diff = diff.total_seconds() / 3600 
                        
                    print(diff)
                    print(e_row.starting_time)
                    print(bo_row.TimeStep)
                    print("-------------------------------------")

In [3]:
bording  = pd.read_csv("Files/BusArrBord/RK_Boardings_2017.csv", sep=';',decimal=',')
alightings  = pd.read_csv("Files/BusArrBord/RK_Alightings_2017.csv", sep=';',decimal=',')

bording['TimeStep']=pd.to_datetime(bording['TimeStep'], format="%Y-%m-%d %H:%M:%S")
alightings['TimeStep']=pd.to_datetime(alightings['TimeStep'], format="%Y-%m-%d %H:%M:%S")

In [4]:
stop_point = pd.read_csv("Files/Poi_StopPoint_1km.csv", sep=';',decimal=',')
venue_poi=stop_point['PoiName'].unique()
stop_point=stop_point[['PoiName', 'StopPointId', 'Distance']]
stop_point=stop_point[stop_point['StopPointId']<8000000]
stop_point=stop_point[stop_point['StopPointId']!=10737]

# selecting the closest stops
list_closest_stops=[]
n=3          #number of closest stops to consider
for ven in venue_poi:
    only_one=stop_point[stop_point['PoiName']==ven]
    only_one.sort_values(by='Distance',inplace=True)
    for i in range(n):
        list_closest_stops.append(only_one.iloc[i])

list_closest_stops_df=pd.DataFrame(list_closest_stops)

# Opening events datasets
events17=pd.read_csv("C:\\Users\\User\\Desktop\\DTU\\Thesis\\Python Files\\Files\\events\\READY\\Event_with_topics_label.csv",encoding='latin-1'      )
events17['starting_time']=pd.to_datetime(events17['starting_time'],format="%Y-%m-%d %H:%M:%S")
events17['end_time']=pd.to_datetime(events17['end_time'],format="%Y-%m-%d %H:%M:%S")

    
#merge bording and stops
bord_st_merged=pd.merge(bording, list_closest_stops_df, how='outer', on='StopPointId')
bord_st_merged=bord_st_merged.dropna(subset=['PoiName'])
bord_st_merged=bord_st_merged[bord_st_merged['BoardingPassengerCount']>0]

ali_st_merged=pd.merge(alightings, list_closest_stops_df, how='outer', on='StopPointId')
ali_st_merged=ali_st_merged.dropna(subset=['PoiName'])
ali_st_merged=ali_st_merged[ali_st_merged['AlightingPassengerCount']>0]

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [6]:
stop_point[stop_point['PoiName']==stop_point.PoiName.unique()[0]]

,PoiName,StopPointId,Distance
0,Bella Center,28000,297.093086
1,Bella Center,28005,299.084817
2,Bella Center,27999,299.715360
3,Bella Center,28004,312.943781
4,Bella Center,28002,317.730171
5,Bella Center,28006,320.321621
6,Bella Center,46742,327.383643
8,Bella Center,52890,337.900355
9,Bella Center,45842,439.282349
10,Bella Center,2747,499.307555


In [36]:
######SELECT
venue_names=['Parken', 'Vega', 'Forum', 'Bella Center', 'Royal Arena','DR Koncerthuset']
venue_names2= [ 'TeliaParken', 'Vega', 'ForumCopenhagen','BellaCenter',  'RoyalArena','DrKoncerthuset']

In [37]:
list_df_per_venue=[]
for ven_nam in venue_names:
    df_per_ven=bord_st_merged[bord_st_merged['PoiName']==ven_nam]
    list_df_per_venue.append(df_per_ven)
    
#create event of event per each venue
list_ev_per_venue=[]
for ven_nam in venue_names2:
    ev_per_ven=events17[events17['venue_unique']==ven_nam]
    list_ev_per_venue.append(ev_per_ven)

In [43]:
events17.columns

Index(['Unnamed: 0', 'end_time', 'eventname', 'starting_time', 'venue_unique',
       'Music', 'Alternative&Punk_Music', 'Business', 'Children',
       'Classic_Music', 'Classical_Music', 'Culture', 'Design', 'Education',
       'Electronica_Music', 'Entertainment', 'Football', 'Fælledparken',
       'Indipendent_Music', 'Lifestyle', 'Medicine', 'Metal_Music', 'Music.1',
       'Pop_Music', 'Religious_Music', 'Rock_Music', 'Sport',
       'StageMusicals_Music', 'Traditional_Music', 'Urban_Music',
       'entertainment'],
      dtype='object')

In [45]:
topics_array=events17[['Music', 'Alternative&Punk_Music', 'Business', 'Children',
       'Classic_Music', 'Classical_Music', 'Culture', 'Design', 'Education',
       'Electronica_Music', 'Entertainment', 'Football', 'Fælledparken',
       'Indipendent_Music', 'Lifestyle', 'Medicine', 'Metal_Music', 'Music.1',
       'Pop_Music', 'Religious_Music', 'Rock_Music', 'Sport',
       'StageMusicals_Music', 'Traditional_Music', 'Urban_Music',
       'entertainment']].values

In [52]:
#create df of event per each venue
list_df_per_venue=[]
for ven_nam in venue_names:
    df_per_ven=bord_st_merged[bord_st_merged['PoiName']==ven_nam]
    list_df_per_venue.append(df_per_ven)
    
#create event of event per each venue
list_ev_per_venue=[]
for ven_nam in venue_names2:
    ev_per_ven=events17[events17['venue_unique']==ven_nam]
    list_ev_per_venue.append(ev_per_ven)

#use the event dataset to check when there are the event
df_ult_list_bo=[]

for i in range(6): ####################change
    df=list_df_per_venue[i]
    ev=list_ev_per_venue[i]
    
    event_1h_bef= np.zeros((len(df,)))
    event_2h_bef= np.zeros((len(df,)))
    event_3h_bef= np.zeros((len(df,)))

    event_during= np.zeros((len(df,)))
    
    event_1h_aft= np.zeros((len(df,)))
    event_2h_aft= np.zeros((len(df,)))
    event_3h_aft= np.zeros((len(df,)))
    
    Time_difference=[9999999999 for x in range(len(df))]
    
    topic=np.zeros((len(df,),26))



    
    print("---------------------------------------------------")
    print(len(df))
    
    df.reset_index(drop=True, inplace=True)

    for bo_index, bo_row in df.iterrows():
        if bo_index %5000==0:
            print(bo_index)
                
        for e_index, e_row in ev.iterrows():
           
            if e_row.starting_time.day == bo_row.TimeStep.day and e_row.starting_time.month == bo_row.TimeStep.month and e_row.starting_time.year == bo_row.TimeStep.year:
                
                diff= e_row.starting_time - bo_row.TimeStep
                diff = diff.total_seconds() / 3600 
                
                if diff >0 and diff < 3:
                    topic[bo_index]=topics_array[e_index]
 
                if Time_difference[bo_index]>diff:
                    Time_difference[bo_index]=diff

                if diff > 0 and diff <= 1:
                    event_1h_bef[bo_index] = 1

                elif diff > 1 and diff <= 2:
                    event_2h_bef[bo_index] = 1

                elif diff > 2 and diff <= 3:
                    event_3h_bef[bo_index] = 1

                diff= e_row.end_time - bo_row.TimeStep
                diff = diff.total_seconds() / 3600 
                
                if diff <=0 and diff >=-3:
                    topic[bo_index]=topics_array[e_index]
                
                if diff <= 0 and diff >=-1:
                    event_1h_aft[bo_index] = 1
                    
                elif diff < -1 and diff >=-2:
                    event_2h_aft[bo_index] = 1
                    
                elif diff < -2 and diff >=-3:
                    event_3h_aft[bo_index] = 1
                
                if bo_row.TimeStep>e_row.starting_time and bo_row.TimeStep<e_row.end_time:
                    event_during[bo_index]=1
                    topic[bo_index]=topics_array[e_index]


    df['event_1h_bef']=event_1h_bef
    df['event_2h_bef']=event_2h_bef
    df['event_3h_bef']=event_3h_bef

    df['event_1h_aft']=event_1h_aft
    df['event_2h_aft']=event_2h_aft
    df['event_3h_aft']=event_3h_aft

    
    df['Time_difference']=Time_difference
    
    df['event_during']=event_during
    
    topicDF=pd.DataFrame(topic,columns=[['Music', 'Alternative&Punk_Music', 'Business', 'Children',
       'Classic_Music', 'Classical_Music', 'Culture', 'Design', 'Education',
       'Electronica_Music', 'Entertainment', 'Football', 'Fælledparken',
       'Indipendent_Music', 'Lifestyle', 'Medicine', 'Metal_Music', 'Music.1',
       'Pop_Music', 'Religious_Music', 'Rock_Music', 'Sport',
       'StageMusicals_Music', 'Traditional_Music', 'Urban_Music',
       'entertainment']])

    df_t=pd.concat([df,topicDF],axis=1)

    
    df_ult_list_bo.append(df_t)  

---------------------------------------------------
7257
0
5000


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-

---------------------------------------------------
24258
0
5000
10000
15000
20000
---------------------------------------------------
25261
0
5000
10000
15000
20000
25000
---------------------------------------------------
20112
0
5000
10000
15000
20000
---------------------------------------------------
6277
0
5000
---------------------------------------------------
12295
0
5000
10000


In [54]:
#lag boarding  by stop
df_all=pd.concat(df_ult_list_bo)
list_stopID=df_all.StopPointId.unique()
list_df_per_stop=[]
for stopID in list_stopID:
    df_per_stop=df_all[df_all['StopPointId']==stopID]
    list_df_per_stop.append(df_per_stop)

list_df_lagged=[]
for df in list_df_per_stop:
    df_lagged=buildLaggedFeatures(df, ['BoardingPassengerCount'])
    list_df_lagged.append(df_lagged)

#put togetther 
df_final=pd.concat(list_df_lagged)
df_final.sort_values(by='TimeStep',inplace=True)
df_final.reset_index(drop=True, inplace=True)

#datatime
df_final["day"] = df_final["TimeStep"].dt.weekday
df_final["hours"] = df_final["TimeStep"].dt.hour
ho=np.array(pd.get_dummies(df_final["hours"]))
ho=pd.DataFrame(ho)
df_final=pd.concat([df_final, ho], axis=1)
df_final['isWeekend']=df_final['day'].apply(lambda x: x>4)
df_final['isWeekend']=df_final['isWeekend']*1

#rename column of hour
dfs_new=[]
dfs_new.append(df_final)
for i in range(24):
    df_old=dfs_new[i]
    df_new=df_old.rename(index=str, columns={i: "h_%s"%(i)})
    dfs_new.append(df_new)
    
fin_df=dfs_new[24]


#save
fin_df.to_pickle("C:\\Users\\User\\Desktop\\DTU\\Thesis\\Python Files\\Files\\FinalDFs\\Boarding_2017")

### Wheather data

In [55]:

col=["Temp. (Â°C)high","Temp. (Â°C)avg","Temp. (Â°C)low","Dew Point (Â°C)high","Dew Point (Â°C)avg","Dew Point (Â°C)low","Humidity (%)high",
"Humidity (%)avg","Humidity (%)low","Sea Level Press. (hPa)high","Sea Level Press. (hPa)avg","Sea Level Press. (hPa)low","Visibility (km)high",
"Visibility (km)avg","Visibility (km)low","Wind (km/h)high","Wind (km/h)avg","Wind (km/h)high","Precip. (mm)","Events"]

weathers=[]
for i in range(12):
    i=i+1
    w=pd.read_csv("C:/Users/User/Desktop/DTU/Thesis/Python Files/Files/Weather data/w%s.csv"%(i), names=col,skiprows=[0,1])
    weathers.append(w)
    
fin_df.reset_index(inplace=True,drop=True)
fin_df['Precipitation']=None
fin_df['Temperature']=None

for ind,row in weather.iterrows():
    month=row.DateTime.month
    day=row.DateTime.day
    fin_df['Precipitation'][(fin_df['TimeStep'].dt.month==month) & (fin_df['TimeStep'].dt.day==day) ]=row['Precip. (mm)'] 
    fin_df['Temperature'][(fin_df['TimeStep'].dt.month==month) & (fin_df['TimeStep'].dt.day==day) ]=row['Temp. (Â°C)avg']        
    if day==1:
        print(month)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1
2
3
4
5
6
7
8
9
10
11
12


In [56]:
fin_df.to_pickle("C:\\Users\\User\\Desktop\\DTU\\Thesis\\Python Files\\Files\\FinalDFs\\Boarding_2017_w")

## Alighting

In [57]:
#create df of event per each venue
list_df_per_venue=[]
for ven_nam in venue_names:
    df_per_ven=ali_st_merged[ali_st_merged['PoiName']==ven_nam]
    list_df_per_venue.append(df_per_ven)

#create event of event per each venue
list_ev_per_venue=[]
for ven_nam in venue_names2:
    ev_per_ven=events17[events17['venue_unique']==ven_nam]
    list_ev_per_venue.append(ev_per_ven)

#use the event dataset to check when there are the event
df_ult_list_al=[]


for i in range(6): ##################change
    df=list_df_per_venue[i]
    ev=list_ev_per_venue[i]
    
    event_1h_bef= np.zeros((len(df,)))
    event_2h_bef= np.zeros((len(df,)))
    event_3h_bef= np.zeros((len(df,)))
    
    event_1h_aft= np.zeros((len(df,)))
    event_2h_aft= np.zeros((len(df,)))
    event_3h_aft= np.zeros((len(df,)))
    
    Time_difference=[9999999999 for x in range(len(df))]
    
    event_during= np.zeros((len(df,)))
    
    topic=np.zeros((len(df,),26))
    
    print("---------------------------------------------------")
    print(len(df))
    
    df.reset_index(drop=True, inplace=True)

    for bo_index, bo_row in df.iterrows():
        if bo_index %5000==0:
            print(bo_index)
                
        for e_index, e_row in ev.iterrows():
           
            if e_row.starting_time.day == bo_row.TimeStep.day and e_row.starting_time.month == bo_row.TimeStep.month and e_row.starting_time.year == bo_row.TimeStep.year:
                
                diff= e_row.starting_time - bo_row.TimeStep
                diff = diff.total_seconds() / 3600 
                
                if diff >0 and diff < 3:
                    topic[bo_index]=topics_array[e_index]
 
                if Time_difference[bo_index]>diff:
                    Time_difference[bo_index]=diff

                if diff > 0 and diff <= 1:
                    event_1h_bef[bo_index] = 1

                elif diff > 1 and diff <= 2:
                    event_2h_bef[bo_index] = 1

                elif diff > 2 and diff <= 3:
                    event_3h_bef[bo_index] = 1

                diff= e_row.end_time - bo_row.TimeStep
                diff = diff.total_seconds() / 3600 
                
                if diff <=0 and diff >=-3:
                    topic[bo_index]=topics_array[e_index]
                
                if diff <= 0 and diff >=-1:
                    event_1h_aft[bo_index] = 1
                    
                elif diff < -1 and diff >=-2:
                    event_2h_aft[bo_index] = 1
                    
                elif diff < -2 and diff >=-3:
                    event_3h_aft[bo_index] = 1
                
                if bo_row.TimeStep>e_row.starting_time and bo_row.TimeStep<e_row.end_time:
                    event_during[bo_index]=1
                    topic[bo_index]=topics_array[e_index]

    df['event_1h_bef']=event_1h_bef
    df['event_2h_bef']=event_2h_bef
    df['event_3h_bef']=event_3h_bef

    df['event_1h_aft']=event_1h_aft
    df['event_2h_aft']=event_2h_aft
    df['event_3h_aft']=event_3h_aft

    
    df['Time_difference']=Time_difference
    
    df['event_during']=event_during
    
    topicDF=pd.DataFrame(topic,columns=[['Music', 'Alternative&Punk_Music', 'Business', 'Children',
       'Classic_Music', 'Classical_Music', 'Culture', 'Design', 'Education',
       'Electronica_Music', 'Entertainment', 'Football', 'Fælledparken',
       'Indipendent_Music', 'Lifestyle', 'Medicine', 'Metal_Music', 'Music.1',
       'Pop_Music', 'Religious_Music', 'Rock_Music', 'Sport',
       'StageMusicals_Music', 'Traditional_Music', 'Urban_Music',
       'entertainment']])

    df_t=pd.concat([df,topicDF],axis=1)

    
    df_ult_list_al.append(df_t)  

---------------------------------------------------
6789
0
5000


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-

---------------------------------------------------
28176
0
5000
10000
15000
20000
25000
---------------------------------------------------
24829
0
5000
10000
15000
20000
---------------------------------------------------
21242
0
5000
10000
15000
20000
---------------------------------------------------
6994
0
5000
---------------------------------------------------
12304
0
5000
10000


In [59]:
#lag dwell second by stop
df_all=pd.concat(df_ult_list_al)
list_stopID=df_all.StopPointId.unique()
list_df_per_stop=[]
for stopID in list_stopID:
    df_per_stop=df_all[df_all['StopPointId']==stopID]
    list_df_per_stop.append(df_per_stop)

list_df_lagged=[]
for df in list_df_per_stop:
    df_lagged=buildLaggedFeatures(df, ['AlightingPassengerCount'])
    list_df_lagged.append(df_lagged)

#put togetther 
df_final=pd.concat(list_df_lagged)
df_final.sort_values(by='TimeStep',inplace=True)
df_final.reset_index(drop=True, inplace=True)

#datatime
df_final["day"] = df_final["TimeStep"].dt.weekday
df_final["hours"] = df_final["TimeStep"].dt.hour
ho=np.array(pd.get_dummies(df_final["hours"]))
ho=pd.DataFrame(ho)
df_final=pd.concat([df_final, ho], axis=1)
df_final['isWeekend']=df_final['day'].apply(lambda x: x>4)
df_final['isWeekend']=df_final['isWeekend']*1

#rename column of hour
dfs_new=[]
dfs_new.append(df_final)
for i in range(24):
    df_old=dfs_new[i]
    df_new=df_old.rename(index=str, columns={i: "h_%s"%(i)})
    dfs_new.append(df_new)
    
fin_df_al=dfs_new[24]

#distance betweeen venue and the nearest train station

#save
fin_df_al.to_pickle("C:\\Users\\User\\Desktop\\DTU\\Thesis\\Python Files\\Files\\FinalDFs\\Alighting_2017")

In [60]:
# Main dataset
f1 = open("C:\\Users\\User\\Desktop\\DTU\\Thesis\\Python Files\\Files\\FinalDFs\\Alighting_2017", 'rb')
fin_df = pickle.load(f1)
f1.close()


In [61]:
fin_df.reset_index(inplace=True,drop=True)
fin_df['Precipitation']=None
fin_df['Temperature']=None

for ind,row in weather.iterrows():
    month=row.DateTime.month
    day=row.DateTime.day
    fin_df['Precipitation'][(fin_df['TimeStep'].dt.month==month) & (fin_df['TimeStep'].dt.day==day) ]=row['Precip. (mm)'] 
    fin_df['Temperature'][(fin_df['TimeStep'].dt.month==month) & (fin_df['TimeStep'].dt.day==day) ]=row['Temp. (Â°C)avg']        
    if day==1:
        print(month)



C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


1
2
3
4
5
6
7
8
9
10
11
12


In [62]:
fin_df.to_pickle("C:\\Users\\User\\Desktop\\DTU\\Thesis\\Python Files\\Files\\FinalDFs\\Alighting_2017_w")